In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2021-03-04 17:14:18,979 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [3]:
%load_ext autoreload 
%autoreload 2
from virasana.integracao.conformidade_alchemy import Conformidade
from virasana.scripts.conformidadeupdate import completa_conformidade, preenche_isocode
engine = create_engine(SQL_URI)
completa_conformidade(db, engine, 10)
preenche_isocode(db, engine, 10)

2021-03-04 17:14:32,554 ajna         INFO     10 análises de conformidade complementadas em 0.7764837741851807 segundos.0.07764837741851807 por registro
2021-03-04 17:14:32,963 ajna         INFO     0 isocode preenchidos em 0.3878445625305176 segundos.0 por registro


# Checagens de conformidade

In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
rs = session.execute('SELECT count(*) FROM dbmercante.ajna_conformidade;')

In [6]:
rs.scalar()

118949

In [7]:
SQL_CONFORMIDADE = '''
select cod_recinto, count(*), avg(height), std(height),
SUM(if(height < 700, 1, 0)) / count(*) * 100 as height_baixo,
avg(ratio), stddev(ratio),  min(ratio), max(ratio),
 SUM(if(ratio < 1.9, 1, 0)) / count(ratio) * 100 as ratio_baixo
 avg()
 from ajna_conformidade group by cod_recinto
 '''
df = pd.read_sql(SQL_CONFORMIDADE, engine)

In [8]:
df

,cod_recinto,count(*),avg(height),std(height),height_baixo,avg(ratio),stddev(ratio),min(ratio),max(ratio),ratio_baixo
0,BANDEIRANTES,1473,799.3530,0.4779,0.0000,2.424963,0.699565,1.00,5.73,22.1996
1,BTP1,26472,799.0281,0.1652,0.0000,2.660038,0.506592,0.92,6.02,4.7220
2,BTP2,12030,799.0273,0.1631,0.0000,2.330207,0.442615,0.93,5.53,14.7631
3,CRAGEA,640,879.9313,0.2530,0.0000,2.173406,0.497891,0.45,3.69,10.4688
4,DEICMAR,3712,799.0709,0.2576,0.0000,1.674068,0.443253,0.56,4.24,77.1282
5,ECOPORTO,4567,798.6173,15.3648,0.0657,2.675640,0.558109,0.04,12.28,3.5472
6,EMBRAPORT,15147,799.3143,0.4642,0.0000,4.614376,0.969757,1.30,10.77,0.1915
7,EUDMARCO,1969,213.2743,110.0341,96.3941,3.963174,0.216554,2.49,4.00,0.0000
8,LOCALFRIO,4994,799.0012,0.0400,0.0000,2.202171,0.221436,0.60,3.52,3.1237
9,MARIMEX,5068,799.0294,0.1689,0.0000,2.562121,0.614017,1.05,5.87,10.7143
